In [1]:
import redis
import hashlib

# Ejemplo de predicción de modelo usando Redis

Este notebook muestra cómo obtener predicciones de un modelo que produce predicciones en lotes. Las predicciones se cargaron en Redis. 

En este notebook, recuperamos las predicciones del modelo almacenadas en Redis. Los datos deben ser leídos, convertidos a cadenas y luego hasheados. Con este enfoque, podemos verificar si los datos existen en Redis y recuperar la predicción correspondiente. En caso de que los datos no existan, asignamos un valor de cero. Esta estrategia simula cómo podría comportarse un servicio en producción ante casos no contemplados.

La ventaja de utilizar Redis en este contexto radica en su capacidad para almacenar datos de forma eficiente en memoria, lo que permite un acceso rápido a las predicciones previamente calculadas. Además, Redis ofrece funcionalidades de almacenamiento en caché y persistencia de datos, lo que garantiza la disponibilidad y la integridad de las predicciones incluso en entornos de producción de alta demanda.

In [2]:
# Conectamos al servidor redis (correr el docker compose previamente, sino no funcionará)
r = redis.Redis(host='localhost', port=6379, decode_responses=True)

In [3]:
# Obtención de datos para prueba
test_values = [
    [4.9, 3.0, 1.4, 0.2],
    [6.2, 2.9, 4.3, 1.3],
    [5.7, 2.9, 4.2, 1.3],
    [4.7, 3.2, 1.3, 0.2],
    [4.6, 3.4, 1.4, 0.3],
    [6.3, 3.4, 5.6, 2.4],
    [5.7, 2.9, 4.2, 1.3],
    [6.9, 3.1, 5.4, 2.1],
    [0, 0, 0, 0],
]

# Conversión de valores a cadenas y hash
# Esto debería implementarse en el pipeline. Dado que los números de punto flotante pueden 
# presentar problemas debido a pequeñas variaciones, se podría considerar redondearlos.
string_values = [' '.join(map(str, sublist)) for sublist in test_values]
hashed_values = [hashlib.sha256(substring.encode()).hexdigest() for substring in string_values]  

# Obtención de las salidas del modelo desde Redis
# Estas salidas ya fueron procesadas en el pasado, por lo que consultamos Redis para obtenerlas.
model_output = [r.get(hash_key) for hash_key in hashed_values]

# Reemplazo de valores nulos con un valor predeterminado
# Esto es necesario porque, en caso de una nueva entrada que el modelo no haya visto durante 
# el procesamiento por lotes, necesitamos proporcionar una salida. Esta entrada podría ser
# encolada para su posterior procesamiento en lotes.
model_output = [value if value is not None else 0 for value in model_output]

Veamos la salida del modelo para diferentes entradas:

In [4]:
for index, test_value in enumerate(test_values):
    print(f"Para la entrada: {test_value}")
    
    print(f"El modelo predice que: {model_output[index]}")

Para la entrada: [4.9, 3.0, 1.4, 0.2]
El modelo predice que: 0
Para la entrada: [6.2, 2.9, 4.3, 1.3]
El modelo predice que: 1
Para la entrada: [5.7, 2.9, 4.2, 1.3]
El modelo predice que: 1
Para la entrada: [4.7, 3.2, 1.3, 0.2]
El modelo predice que: 0
Para la entrada: [4.6, 3.4, 1.4, 0.3]
El modelo predice que: 0
Para la entrada: [6.3, 3.4, 5.6, 2.4]
El modelo predice que: 2
Para la entrada: [5.7, 2.9, 4.2, 1.3]
El modelo predice que: 1
Para la entrada: [6.9, 3.1, 5.4, 2.1]
El modelo predice que: 2
Para la entrada: [0, 0, 0, 0]
El modelo predice que: 0
